In [ ]:
import requests
import statistics
from math import sqrt

def get_public_ip():
    try:
        ip = requests.get('https://api.ipify.org').text
        print("Your Public IP Address is:", ip)
    except Exception as e:
        print("Error retrieving IP address:", e)

get_public_ip()

def get_ip_api():
    try:
        r = requests.get("http://ip-api.com/json/").json()
        return {
            "source": "ip-api.com",
            "lat": float(r['lat']),
            "lon": float(r['lon']),
            "city": r.get("city"),
            "region": r.get("regionName"),
            "country": r.get("country"),
            "zip": r.get("zip"),
            "isp": r.get("isp")
        }
    except:
        return None

def get_ipinfo():
    try:
        r = requests.get("https://ipinfo.io/json").json()
        lat, lon = map(float, r['loc'].split(','))
        return {
            "source": "ipinfo.io",
            "lat": lat,
            "lon": lon,
            "city": r.get("city"),
            "region": r.get("region"),
            "country": r.get("country"),
            "zip": r.get("postal"),
            "isp": r.get("org")
        }
    except:
        return None

def get_geojs():
    try:
        r = requests.get("https://get.geojs.io/v1/ip/geo.json").json()
        return {
            "source": "geojs.io",
            "lat": float(r['latitude']),
            "lon": float(r['longitude']),
            "city": r.get("city"),
            "region": r.get("region"),
            "country": r.get("country"),
            "zip": r.get("postal_code"),
            "isp": r.get("organization_name")
        }
    except:
        return None

def distance(p1, p2):
    return sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

# Collect results
results = []
for func in [get_ip_api, get_ipinfo, get_geojs]:
    res = func()
    if res:
        results.append(res)

# Print raw sources
print("\nGeolocation Sources:")
for r in results:
    print(f"\nSource: {r['source']}")
    print(f"Location: {r.get('city')}, {r.get('region')}, {r.get('country')} {r.get('zip')}")
    print(f"Latitude: {r['lat']:.5f}, Longitude: {r['lon']:.5f}")
    print(f"ISP: {r.get('isp')}")

# Calculate average coordinates
if results:
    lats = [r['lat'] for r in results]
    lons = [r['lon'] for r in results]
    avg_lat = statistics.mean(lats)
    avg_lon = statistics.mean(lons)

    # Confidence calculation
    dists = []
    for i in range(len(results)):
        for j in range(i+1, len(results)):
            dists.append(distance((results[i]['lat'], results[i]['lon']), (results[j]['lat'], results[j]['lon'])))

    avg_dist = statistics.mean(dists) if dists else 0
    if avg_dist < 0.01:
        confidence = "HIGH"
    elif avg_dist < 0.1:
        confidence = "MEDIUM"
    else:
        confidence = "LOW"

    print(f"\nEstimated Average Location:\nLatitude: {avg_lat:.5f}, Longitude: {avg_lon:.5f}")
    print(f"Confidence Score: {confidence}")
else:
    print("\nError: No geolocation sources responded.")
